In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data.csv', delimiter=';')

In [3]:
# Check for any object type columns in the dataframe
object_columns = data.select_dtypes(include=['object']).columns

print(object_columns)

Index(['Target'], dtype='object')


In [4]:
from sklearn.preprocessing import LabelEncoder

# Convert to correct data structure
label_encoder = LabelEncoder()

data['Target'] = label_encoder.fit_transform(data['Target'])

In [5]:
X = data.drop('Target', axis=1).values
y = data['Target'].values

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
class SimpleNN(nn.Module):
    def __init__(self, input_features, output_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_features, 50)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(50, output_classes)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out

In [7]:
input_features = X_train.shape[1]
output_classes = len(np.unique(y))

# Initialize the model, loss function, and optimizer
model = SimpleNN(input_features, output_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 20
for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/20, Loss: 0.9190513491630554
Epoch 2/20, Loss: 0.9137364625930786
Epoch 3/20, Loss: 0.6911916136741638
Epoch 4/20, Loss: 0.8763297200202942
Epoch 5/20, Loss: 0.8505054116249084
Epoch 6/20, Loss: 0.7109635472297668
Epoch 7/20, Loss: 0.7374888062477112
Epoch 8/20, Loss: 0.7471256852149963
Epoch 9/20, Loss: 0.6297647953033447
Epoch 10/20, Loss: 0.7460622787475586
Epoch 11/20, Loss: 0.6823877692222595
Epoch 12/20, Loss: 0.9147528409957886
Epoch 13/20, Loss: 0.6974675059318542
Epoch 14/20, Loss: 0.6570839881896973
Epoch 15/20, Loss: 0.6744030117988586
Epoch 16/20, Loss: 0.6557066440582275
Epoch 17/20, Loss: 0.6511663198471069
Epoch 18/20, Loss: 0.75215744972229
Epoch 19/20, Loss: 0.6141546368598938
Epoch 20/20, Loss: 0.8004282116889954


In [8]:
# Evaluate the model on the test dataset
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy of the model on the test set: {accuracy * 100:.2f}%')

Accuracy of the model on the test set: 75.71%
